# Week 2: Quantitative Measures of Population Health — DALYs and QALYs

**Learning Objectives:**
- Understand the components of Disability-Adjusted Life Years (DALYs)
- Calculate Years of Life Lost (YLL) and Years Lived with Disability (YLD)
- Explore how disability weights are determined and their impact on burden estimates
- Compare DALYs and Quality-Adjusted Life Years (QALYs)
- Critically evaluate the assumptions embedded in these measures

---

## 1. Why Do We Need Summary Measures of Health?

Traditional health statistics focus on **mortality** — death rates, life expectancy, cause of death. But this misses something important: **morbidity**.

Consider two conditions:
- **Ischaemic heart disease**: High mortality, significant disability before death
- **Low back pain**: Very low mortality, but affects millions and causes significant suffering

If we only count deaths, we systematically undervalue conditions that cause suffering without killing. Summary measures like DALYs and QALYs attempt to capture **both** mortality and morbidity in a single metric.

### The Policy Question

Imagine you're advising a health ministry with a fixed budget. How do you compare:
- Preventing 100 deaths from heart disease?
- Preventing 10,000 cases of chronic low back pain?
- Curing 500 cases of blindness?

To make these comparisons, we need a **common currency** for health.

## 2. Setup

First, we'll load the utility functions and data for this course.

In [ ]:
# ============================================================
# Bootstrap cell (works both locally and in Colab)
#
# What this cell does:
# - Ensures that we are inside the course repository.
# - In Colab: clones the repository from GitHub if necessary.
# - Loads the course utility module (epi_utils.py).
#
# Important:
# - You may see messages printed below (e.g. from pip or git).
# - Warnings (often in yellow) are usually harmless.
# - If you see a red error traceback, re-run this cell first.
# ============================================================

import os
import sys
import pathlib
import subprocess

# ------------------------------------------------------------
# Configuration: repository location and URL
# ------------------------------------------------------------
REPO_URL = "https://github.com/ggkuhnle/phn-epi.git"
REPO_DIR = "phn-epi"

# ------------------------------------------------------------
# 1. Ensure we are inside the repository
# ------------------------------------------------------------
cwd = pathlib.Path.cwd()

# Case A: we are already in the repository (scripts/epi_utils.py exists)
if (cwd / "scripts" / "epi_utils.py").is_file():
    repo_root = cwd
# Case B: we are in a subdirectory of the repository
elif (cwd.parent / "scripts" / "epi_utils.py").is_file():
    repo_root = cwd.parent
# Case C: we are outside the repository (e.g. in Colab)
else:
    repo_root = cwd / REPO_DIR

    # Clone the repository if not present
    if not repo_root.is_dir():
        print(f"Cloning repository from {REPO_URL} into {repo_root} ...")
        subprocess.run(["git", "clone", REPO_URL, str(repo_root)], check=True)
    else:
        print(f"Using existing repository at {repo_root}")

    # Change working directory to repository root
    os.chdir(repo_root)
    repo_root = pathlib.Path.cwd()

# Add scripts directory to Python path
scripts_dir = repo_root / "scripts"
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

print(f"Repository root: {repo_root}")
print("Bootstrap completed successfully.")

In [ ]:
# ------------------------------------------------------------
# Import libraries and course utilities
# ------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, VBox, HBox, Output
import ipywidgets as widgets
from IPython.display import display, Markdown

# Import course utilities from the repository
from epi_utils import (
    LIFE_TABLE, GBD_DISABILITY_WEIGHTS, EXERCISE_CONDITIONS,
    get_life_expectancy, calculate_yll, calculate_yld, 
    calculate_dalys, calculate_qalys_gained
)

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]

print("Libraries loaded successfully.")

In [ ]:
# View the reference life table
print("GBD 2019 Reference Life Table")
print("=" * 40)
display(LIFE_TABLE)

In [ ]:
# View disability weights for selected conditions
print("GBD 2019 Disability Weights (selected conditions)")
print("=" * 60)
display(GBD_DISABILITY_WEIGHTS.sort_values('disability_weight', ascending=False))

## 3. Building DALYs from First Principles

The DALY combines two components:

$$\text{DALY} = \text{YLL} + \text{YLD}$$

Where:
- **YLL** (Years of Life Lost) = years lost due to premature mortality
- **YLD** (Years Lived with Disability) = years lived in less than perfect health

**Key insight**: 1 DALY = 1 year of healthy life lost. Higher DALYs = greater burden.

### 3.1 Years of Life Lost (YLL)

YLL measures premature mortality by comparing actual age at death to a reference life expectancy:

$$\text{YLL} = N \times L$$

Where:
- $N$ = number of deaths
- $L$ = standard life expectancy at age of death

In [ ]:
# Example: Mortality from ischaemic heart disease
ihd_deaths = {
    45: 50,
    55: 200,
    65: 500,
    75: 800,
    85: 400
}

total_yll, yll_breakdown = calculate_yll(ihd_deaths)

print(f"Ischaemic Heart Disease - YLL Calculation")
print("=" * 50)
display(yll_breakdown)
print(f"\nTotal Deaths: {sum(ihd_deaths.values()):,}")
print(f"Total YLL: {total_yll:,.0f} years")
print(f"Average YLL per death: {total_yll/sum(ihd_deaths.values()):.1f} years")

### Exercise 3.1: Calculate YLL for a different condition

Road traffic accidents tend to affect younger people. Calculate the YLL using the mortality data below and compare to IHD.

In [ ]:
# Road traffic accident deaths
rta_deaths = {
    15: 30,
    25: 80,
    35: 60,
    45: 40,
    55: 20,
    65: 10
}

# YOUR CODE HERE
# Calculate YLL for road traffic accidents
# Compare total deaths and total YLL between IHD and RTA



### 3.2 Years Lived with Disability (YLD)

YLD captures the morbidity burden:

$$\text{YLD} = P \times DW$$

Where:
- $P$ = prevalence (number of cases) × average duration
- $DW$ = disability weight (0 to 1)

In [ ]:
# Example: Comparing YLD for different conditions
conditions = [
    {'name': 'Type 2 diabetes (uncomplicated)', 'prevalence': 100000, 'dw': 0.015},
    {'name': 'Moderate depression', 'prevalence': 20000, 'dw': 0.145},
    {'name': 'Severe low back pain', 'prevalence': 15000, 'dw': 0.325},
    {'name': 'Blindness', 'prevalence': 5000, 'dw': 0.187},
]

print("YLD Comparison (prevalence-based, 1 year)")
print("=" * 70)

yld_results = []
for c in conditions:
    yld = calculate_yld(c['prevalence'], c['dw'])
    yld_results.append({
        'Condition': c['name'],
        'Prevalence': f"{c['prevalence']:,}",
        'Disability Weight': c['dw'],
        'YLD': round(yld, 0)
    })

display(pd.DataFrame(yld_results))

### 3.3 Putting it Together: Total DALYs

In [ ]:
# Ischaemic Heart Disease: high mortality, moderate disability
ihd = calculate_dalys(
    deaths_by_age={45: 50, 55: 200, 65: 500, 75: 800, 85: 400},
    prevalence=50000,
    disability_weight=0.072,
    condition_name="Ischaemic Heart Disease"
)

In [ ]:
# Low Back Pain: very low mortality, high disability burden
lbp = calculate_dalys(
    deaths_by_age={75: 5, 85: 10},
    prevalence=200000,
    disability_weight=0.325,
    condition_name="Severe Low Back Pain"
)

## 4. The Disability Weight Exercise

### 4.1 How GBD Determines Disability Weights

The Global Burden of Disease study uses **paired comparisons** from population surveys. Respondents are presented with two hypothetical health states and asked which they consider healthier.

### 4.2 Your Turn: Set Your Own Disability Weights

Assign your own disability weights to a set of conditions, then compare to GBD values.

In [ ]:
# Display conditions for students to consider
print("DISABILITY WEIGHT EXERCISE")
print("=" * 80)
print("\nAssign a disability weight (0-1) to each condition below.")
print("0 = perfect health, 1 = equivalent to death")
print("-" * 80)

for i, c in enumerate(EXERCISE_CONDITIONS, 1):
    print(f"\n{i}. {c['name'].upper()}")
    print(f"   {c['description']}")

In [ ]:
# Interactive widget for setting disability weights
sliders = {}
slider_widgets = []

for c in EXERCISE_CONDITIONS:
    slider = FloatSlider(
        value=0.1, min=0, max=1, step=0.01,
        description='',
        continuous_update=False,
        readout_format='.2f',
        layout=widgets.Layout(width='300px')
    )
    sliders[c['name']] = slider
    label = widgets.HTML(value=f"<b>{c['name']}</b>", layout=widgets.Layout(width='200px'))
    slider_widgets.append(HBox([label, slider]))

output = Output()

def compare_weights(change=None):
    with output:
        output.clear_output(wait=True)
        
        comparison_data = []
        for c in EXERCISE_CONDITIONS:
            your_weight = sliders[c['name']].value
            gbd_weight = c['gbd_weight']
            comparison_data.append({
                'Condition': c['name'],
                'Your Weight': your_weight,
                'GBD Weight': gbd_weight,
                'Difference': your_weight - gbd_weight
            })
        
        df = pd.DataFrame(comparison_data)
        
        # Create comparison plot
        fig, ax = plt.subplots(figsize=(12, 6))
        x = np.arange(len(df))
        width = 0.35
        
        ax.bar(x - width/2, df['Your Weight'], width, label='Your weights', color='steelblue')
        ax.bar(x + width/2, df['GBD Weight'], width, label='GBD weights', color='coral')
        
        ax.set_ylabel('Disability Weight')
        ax.set_title('Your Disability Weights vs GBD 2019')
        ax.set_xticks(x)
        ax.set_xticklabels(df['Condition'], rotation=45, ha='right')
        ax.legend()
        ax.set_ylim(0, 1)
        plt.tight_layout()
        plt.show()
        
        print("\nComparison Table:")
        display(df.style.format({
            'Your Weight': '{:.3f}',
            'GBD Weight': '{:.3f}',
            'Difference': '{:+.3f}'
        }).background_gradient(subset=['Difference'], cmap='RdYlGn_r', vmin=-0.3, vmax=0.3))

for slider in sliders.values():
    slider.observe(compare_weights, names='value')

compare_button = widgets.Button(description="Compare to GBD", button_style='primary')
compare_button.on_click(compare_weights)

print("Adjust the sliders to set your disability weights, then click 'Compare to GBD':")
display(VBox(slider_widgets + [compare_button, output]))

### 4.3 Discussion Questions

1. **Where did you differ most from GBD?** Why might your valuation differ?

2. **Adaptation**: People living with a condition often rate it less severe. Whose values should count?

3. **Cultural variation**: Might conditions be valued differently across cultures?

4. **The "worse than death" problem**: GBD caps weights at 1.0. How should we handle states some consider worse than death?

## 5. How Disability Weights Change Rankings

Let's see how your disability weights would change burden rankings.

In [ ]:
# Hypothetical prevalence data
population_data = {
    'Mild anaemia': {'prevalence': 500000, 'deaths_by_age': {}},
    'Moderate hearing loss': {'prevalence': 300000, 'deaths_by_age': {}},
    'Moderate depression': {'prevalence': 150000, 'deaths_by_age': {45: 50, 55: 30, 65: 20}},
    'Severe low back pain': {'prevalence': 100000, 'deaths_by_age': {}},
    'Complete blindness': {'prevalence': 30000, 'deaths_by_age': {}},
    'Severe dementia': {'prevalence': 80000, 'deaths_by_age': {75: 2000, 85: 5000}},
    'Type 2 diabetes (controlled)': {'prevalence': 400000, 'deaths_by_age': {55: 100, 65: 500, 75: 1000, 85: 500}},
    'Obesity (BMI ≥ 40)': {'prevalence': 200000, 'deaths_by_age': {45: 50, 55: 200, 65: 300, 75: 200}}
}

def calculate_burden_rankings(population_data, disability_weights):
    results = []
    for condition, data in population_data.items():
        dw = disability_weights.get(condition, 0.1)
        yll = 0
        if data['deaths_by_age']:
            yll, _ = calculate_yll(data['deaths_by_age'])
        yld = calculate_yld(data['prevalence'], dw)
        dalys = yll + yld
        results.append({'Condition': condition, 'DW': dw, 'YLL': yll, 'YLD': yld, 'DALYs': dalys})
    
    df = pd.DataFrame(results).sort_values('DALYs', ascending=False)
    df['Rank'] = range(1, len(df) + 1)
    return df[['Rank', 'Condition', 'DW', 'YLL', 'YLD', 'DALYs']]

In [ ]:
# Calculate rankings with GBD weights
gbd_weights = {c['name']: c['gbd_weight'] for c in EXERCISE_CONDITIONS}
gbd_rankings = calculate_burden_rankings(population_data, gbd_weights)

print("Disease Burden Rankings using GBD Disability Weights")
print("=" * 80)
display(gbd_rankings.style.format({'DW': '{:.3f}', 'YLL': '{:,.0f}', 'YLD': '{:,.0f}', 'DALYs': '{:,.0f}'}))

In [ ]:
# Calculate rankings with YOUR weights
your_weights = {name: slider.value for name, slider in sliders.items()}
your_rankings = calculate_burden_rankings(population_data, your_weights)

print("Disease Burden Rankings using YOUR Disability Weights")
print("=" * 80)
display(your_rankings.style.format({'DW': '{:.3f}', 'YLL': '{:,.0f}', 'YLD': '{:,.0f}', 'DALYs': '{:,.0f}'}))

## 6. DALYs vs QALYs

| Aspect | DALY | QALY |
|--------|------|------|
| **Direction** | Higher = more burden | Higher = more health |
| **Primary use** | Population burden estimation | Cost-effectiveness analysis |
| **Key user** | GBD, WHO | NICE, health economists |

In [ ]:
# Example: Cataract surgery
qalys, utility_gain = calculate_qalys_gained(
    intervention_effect=0.95,
    population=10000,
    duration=15,
    dw_before=0.187,
    dw_after=0.003
)

print("Cataract Surgery Programme")
print("=" * 50)
print(f"Population: 10,000 people with cataracts")
print(f"Success rate: 95%")
print(f"Duration of benefit: 15 years")
print(f"\nUtility gain per person: {utility_gain:.3f}")
print(f"Total QALYs gained: {qalys:,.0f}")
print(f"\nIf programme costs £10M: £{10000000/qalys:,.0f} per QALY")
print(f"NICE threshold: £20,000-£30,000 per QALY")

## 7. Limitations and Critique

### The Disability Rights Critique

Some argue that QALYs/DALYs discriminate against disabled people.

In [ ]:
# Illustration: The discrimination problem
print("The QALY Discrimination Problem")
print("=" * 60)
print("\nScenario: A life-saving treatment adding 10 years of life.\n")

qalys_nondisabled = 10 * (1 - 0)
print(f"Patient A (no disability): 10 years × 1.0 = {qalys_nondisabled:.1f} QALYs\n")

qalys_disabled = 10 * (1 - 0.133)
print(f"Patient B (paraplegia, DW=0.133): 10 years × 0.867 = {qalys_disabled:.1f} QALYs\n")

print("-" * 60)
print(f"Patient A's life is valued {qalys_nondisabled/qalys_disabled:.1%} as much as Patient B's.")
print(f"\nIs this fair?")

## 8. Exercises

### Exercise 1: Calculate DALYs for Iron Deficiency Anaemia

Data (hypothetical UK population):
- Mild anaemia: 800,000 (DW=0.004)
- Moderate anaemia: 150,000 (DW=0.052)
- Severe anaemia: 20,000 (DW=0.149)
- Deaths: 500 at age 75, 1000 at age 85

In [ ]:
# YOUR CODE HERE



### Exercise 2: Reflection Questions

1. The GBD weight for obesity (0.086) is lower than for depression (0.145). Do you agree?

2. Should DALYs include age-weighting (valuing years in middle-age more)? Why did GBD remove this?

3. How might you measure the burden of poor diet without relying on disease categories?

---

## References

- GBD 2019 Diseases and Injuries Collaborators. (2020). *The Lancet*.
- Salomon JA et al. (2015). Disability weights for GBD 2013. *Lancet Global Health*.
- Nord E. (2015). Limitations of the QALY. *Cambridge Quarterly of Healthcare Ethics*.